## Import Library dan Load Dataset

In [3]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, f1_score
from sklearn.model_selection import LeaveOneOut
import matplotlib.pyplot as plt

## Memuat dataset EMNIST

In [5]:
from tensorflow.keras.datasets import mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 5s 0us/step


## Normalisasi data

In [7]:
x_train = x_train / 255.0
x_test = x_test / 255.0
x_train = x_train[..., np.newaxis]
x_test = x_test[..., np.newaxis]

In [9]:
print(f"Bentuk data training: {x_train.shape}")
print(f"Bentuk data testing: {x_test.shape}")

Bentuk data training: (60000, 28, 28, 1)
Bentuk data testing: (10000, 28, 28, 1)


## Fungsi untuk Membuat Model CNN

## Mendefinisikan model CNN

In [11]:
def create_cnn_model(input_shape, num_classes):
    model = models.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),  # Layer konvolusi pertama
        layers.MaxPooling2D((2, 2)),  # Layer pooling pertama
        layers.Conv2D(64, (3, 3), activation='relu'),  # Layer konvolusi kedua
        layers.MaxPooling2D((2, 2)),  # Layer pooling kedua
        layers.Flatten(),  # Flattening
        layers.Dense(128, activation='relu'),  # Fully connected layer
        layers.Dense(num_classes, activation='softmax'),  # Output layer
    ])
    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    return model

## Inisialisasi parameter model

In [13]:
input_shape = x_train.shape[1:]
num_classes = len(np.unique(y_train))

In [15]:
print(f"Bentuk input: {input_shape}")
print(f"Jumlah kelas: {num_classes}")

Bentuk input: (28, 28, 1)
Jumlah kelas: 10


## Leave-One-Out Cross-Validation (LOOCV)

In [26]:
loo = LeaveOneOut()
y_true = []  # Menyimpan label sebenarnya
y_pred = []  # Menyimpan prediksi model

# Loop untuk proses LOOCV
for train_index, test_index in loo.split(x_train[:100]):  # Menggunakan subset data untuk alasan praktis
    x_train_fold, x_test_fold = x_train[train_index], x_train[test_index]
    y_train_fold, y_test_fold = y_train[train_index], y_train[test_index]

    # Membuat dan melatih model
    model = create_cnn_model(input_shape, num_classes)
    model.fit(x_train_fold, y_train_fold, epochs=1, verbose=0)  # Sesuaikan jumlah epoch untuk pelatihan sebenarnya
    y_pred_fold = np.argmax(model.predict(x_test_fold), axis=1)  # Prediksi untuk sampel uji
    
    # Menyimpan hasil prediksi
    y_true.append(y_test_fold[0])
    y_pred.append(y_pred_fold[0])

print("Proses LOOCV selesai!")

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step   
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step   
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 487ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 558ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 573ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 534ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 574ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 562ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 568ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 526ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 725ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 626ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 537ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 531ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 529ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 541ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 567ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 501ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 678ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step   
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 709ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 554ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 717ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 549ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 553ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 

## Mengevaluasi hasil

In [28]:
conf_matrix = confusion_matrix(y_true, y_pred)  # Matriks kebingungan
accuracy = accuracy_score(y_true, y_pred)  # Akurasi
precision = precision_score(y_true, y_pred, average='weighted')  # Presisi
f1 = f1_score(y_true, y_pred, average='weighted')  # F1-Score

C:\Users\HP\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [30]:
conf_matrix = confusion_matrix(y_true, y_pred)  # Matriks kebingungan
accuracy = accuracy_score(y_true, y_pred)  # Akurasi
precision = precision_score(y_true, y_pred, average='weighted', zero_division=1)  # Presisi
f1 = f1_score(y_true, y_pred, average='weighted', zero_division=1)  # F1-Score

## Menampilkan metrik evaluasi

In [48]:
print("Confusion Matrix:")
print(conf_matrix)
print(f"Akurasi: {accuracy:.2f}")
print(f"Presisi: {precision:.2f}")
print(f"F1-Score: {f1:.2f}")

Confusion Matrix:
[[13  0  0  0  0  0  0  0  0  0]
 [ 3  7  0  1  1  0  0  0  0  2]
 [ 5  0  0  1  0  0  0  0  0  0]
 [ 6  1  0  3  0  0  1  0  0  0]
 [ 5  1  0  1  3  0  0  0  0  1]
 [ 3  0  0  1  1  0  0  0  0  0]
 [ 8  0  0  1  0  0  2  0  0  0]
 [ 6  1  0  0  1  0  0  0  1  1]
 [ 3  2  0  0  0  0  1  0  0  2]
 [ 8  0  0  1  1  0  0  0  0  1]]
Akurasi: 0.29
Presisi: 0.47
F1-Score: 0.23
